In [2]:
import pandas as pd
import plotly.express as px
from os import path

In [6]:
df = pd.read_csv('simulations_assigned_output.csv', sep=';', header=None)
day_plan_cnt = 100
df.columns = ['simulation_key', 'missing_plan_cnt_first_delay', 'missing_plan_cnt_planned_time', 'whole_process_delay']
df['controller_name'] = df.simulation_key.str.split('-', expand=True)[0]
df['break_strategy'] = df.simulation_key.str.split('-', expand=True)[1]
df['experiment_name'] = df.controller_name + '-' + df.break_strategy
df['simulation_index'] = df.simulation_key.str.split('-', expand=True)[2].astype(int)
df['produced_in_planned_time'] = day_plan_cnt - df.missing_plan_cnt_planned_time
df['whole_process_delay'] = df.whole_process_delay.str.replace(',', '.').astype(float)
df['process_delay_minutes'] = df.whole_process_delay / 60
df['process_delay_hours'] = df.process_delay_minutes / 60

df

,simulation_key,missing_plan_cnt_first_delay,missing_plan_cnt_planned_time,whole_process_delay,controller_name,break_strategy,experiment_name,simulation_index,produced_in_planned_time,process_delay_minutes,process_delay_hours
0,naive-skip_break-1,75,9,451.36,naive,skip_break,naive-skip_break,1,91,7.522667,0.125378
1,naive-reorganization-1,67,8,432.10,naive,reorganization,naive-reorganization,1,92,7.201667,0.120028
2,async-skip_break-1,39,2,0.80,async,skip_break,async-skip_break,1,98,0.013333,0.000222
3,async-reorganization-1,39,2,0.80,async,reorganization,async-reorganization,1,98,0.013333,0.000222
4,naive-skip_break-2,0,0,0.00,naive,skip_break,naive-skip_break,2,100,0.000000,0.000000
5,naive-reorganization-2,0,0,0.00,naive,reorganization,naive-reorganization,2,100,0.000000,0.000000
6,async-skip_break-2,0,0,0.00,async,skip_break,async-skip_break,2,100,0.000000,0.000000
7,async-reorganization-2,0,0,0.00,async,reorganization,async-reorganization,2,100,0.000000,0.000000
8,naive-skip_break-3,82,17,1198.70,naive,skip_break,naive-skip_break,3,83,19.978333,0.332972
9,naive-reorganization-3,80,17,1195.75,naive,reorganization,naive-reorganization,3,83,19.929167,0.332153


In [7]:
df_summaries = pd.DataFrame(index=df.experiment_name.unique(),
                            columns=['perc_complete_plans', 'perc_zero_delay_plans',
                                    'avg_missing_steps', 'avg_accumulated_delay_minutes',
                                    'missing_steps_90_percentile', 'missing_steps_95_percentile', 'missing_steps_99_percentile',
                                    'missing_steps_70_percentile', 'missing_steps_80_percentile'])

for group_name, df_group in df.groupby('experiment_name'):
    df_summaries.loc[group_name, 'perc_complete_plans'] = df_group[df_group.missing_plan_cnt_planned_time == 0].shape[0] / df_group.shape[0] * 100
    df_summaries.loc[group_name, 'perc_complete_plans'] = str(round(df_summaries.loc[group_name, 'perc_complete_plans'], 1)) + ' %'
    df_summaries.loc[group_name, 'perc_zero_delay_plans'] = df_group[df_group.whole_process_delay < 10].shape[0] / df_group.shape[0] * 100
    df_summaries.loc[group_name, 'perc_zero_delay_plans'] = str(round(df_summaries.loc[group_name, 'perc_zero_delay_plans'], 1)) + ' %'
    df_summaries.loc[group_name, 'avg_missing_steps'] = df_group.missing_plan_cnt_planned_time.mean()
    df_summaries.loc[group_name, 'avg_accumulated_delay_minutes'] = df_group.process_delay_minutes.mean()
    df_summaries.loc[group_name, 'missing_steps_90_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.9)
    df_summaries.loc[group_name, 'missing_steps_95_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.95)
    df_summaries.loc[group_name, 'missing_steps_99_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.99)
    df_summaries.loc[group_name, 'missing_steps_70_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.70)
    df_summaries.loc[group_name, 'missing_steps_80_percentile'] = df_group.missing_plan_cnt_planned_time.quantile(q=0.80)

df_summaries

,perc_complete_plans,perc_zero_delay_plans,avg_missing_steps,avg_accumulated_delay_minutes,missing_steps_90_percentile,missing_steps_95_percentile,missing_steps_99_percentile,missing_steps_70_percentile,missing_steps_80_percentile
naive-skip_break,33.3 %,33.3 %,8.66667,9.167,15.4,16.2,16.84,12.2,13.8
naive-reorganization,33.3 %,33.3 %,8.33333,9.04361,15.2,16.1,16.82,11.6,13.4
async-skip_break,33.3 %,100.0 %,2,0.0544444,3.6,3.8,3.96,2.8,3.2
async-reorganization,33.3 %,100.0 %,2,0.0544444,3.6,3.8,3.96,2.8,3.2
